In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import linearmodels as lm
import matplotlib
import math
import statsmodels.formula.api as smf

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import FirstDifferenceOLS
from linearmodels.panel import compare
from matplotlib import pyplot as plt

from pandas.api.types import is_numeric_dtype

pd.set_option('display.max_columns', 500)

%matplotlib inline

In [27]:
# read the main data set
df=pd.read_csv('merged_df_3.csv',
                    encoding='utf-8')


In [28]:
# exclude if ridership is zero--missing data
df = df[df['UPT']>0]
df = df[df['VRM']>0]

In [29]:
# separate Bus, Rail, DemandResponsive and Other
bus = df[df['Mode']=='Bus']
rail = df[df['Mode']=='Rail']


In [30]:
# set the indices
bus=bus.set_index(['CBSA','Year'])
rail=rail.set_index(['CBSA','Year'])

In [31]:
# keep only the numeric columns -- the estimation will give an error otherwise
bus = bus.select_dtypes(include=[np.number])
rail = rail.select_dtypes(include=[np.number])

In [32]:
# create a log of all fields
for col in bus.columns:
    bus[col+'_log'] = np.log(bus[col]+1)
    
for col in rail.columns:
    rail[col+'_log'] = np.log(rail[col]+1)    

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: invalid value encountered in log
  app.launch_new_instance()
C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: invalid value encountered in log


In [33]:
# check the data
bus.head()

Unnamed: 0  NEW_CBSA       UPT       VRM   UPT_ADJ   VRM_ADJ  \
CBSA  Year                                                                 
10180 2002           0       NaN  493073.0  389678.0  493073.0  389678.0   
      2003           1       NaN  475747.0  383239.0  475747.0  383239.0   
      2004           2       NaN  469981.0  388907.0  469981.0  388907.0   
      2005           3       NaN  507165.0  444176.0  507165.0  444176.0   
      2006           4       NaN  493638.0  441300.0  493638.0  441300.0   

            VRH_ADJ  FARE_TOTAL  FARE_per_UPT  AREALANDPT      Tot_Pop  \
CBSA  Year                                                               
10180 2002  29016.0    162609.0      0.329787   141756054  117258.2188   
      2003  28597.0    173563.0      0.364822   141756054  127098.1250   
      2004  29126.0    185046.0      0.393731   141756054  137113.5000   
      2005  31147.0    213965.0      0.421884   141756054  146427.0000   
      2006  30750.0    201795.0      0.408791   141756054  158548.0000   

            Tot_Instate_Pop  Tot_Outstate_Pop  Tot_NonUSA_POP  \
CBSA  Year                                                      
10180 2002      87723.96875       23752.17188     2987.484375   
      2003      93677.12500       26964.31250     2834.062500   
      2004      99787.50000       30227.75000     2648.750000   
      2005     105269.00000       33286.00000     2591.000000   
      2006     113266.00000       37165.00000     2023.000000   

            Total_Median_Income_Individual  Native_Instate_Med_Inc_Indiv  \
CBSA  Year                                                                 
10180 2002                     19756.67188                   22848.10938   
      2003                     19941.31250                   21972.56250   
      2004                     20132.75000                   21085.75000   
      2005                     20297.00000                   20244.00000   
      2006                     20570.00000                   19222.00000   

            Native_Outstate_Med_Inc_Indiv  Total_Pop_Poverty  \
CBSA  Year                                                     
10180 2002                    17091.51563          147631.94   
      2003                    18472.93750          147198.25   
      2004                    19808.25000          146801.00   
      2005                    21328.00000          146258.00   
      2006                    22110.00000          146298.00   

            Pop_Below100_Poverty  Pop_Below150_Poverty  Pop_Above150_Poverty  \
CBSA  Year                                                                     
10180 2002                 22.30                 12.89                 64.82   
      2003                 20.72                 12.86                 66.43   
      2004                 19.13                 12.88                 68.00   
      2005                 17.60                 12.70                 69.70   
      2006                 15.80                 13.30                 70.90   

            Age_under18  Age_18to64  Age_over64  Total_HH  HH_0Veh   HH_1Veh  \
CBSA  Year                                                                     
10180 2002        31.23       54.90       13.87  61601.69  1195.73  27303.56   
      2003        29.78       56.41       13.81  61229.25  1515.06  25910.75   
      2004        28.30       57.95       13.75  60833.00  1823.75  24514.00   
      2005        26.90       59.40       13.70  60532.00  2175.00  23133.00   
      2006        25.20       61.20       13.60  59850.00  2356.00  21689.00   

             HH_2Veh  HH_3Veh  HH_4+Veh  PCT_HH_NO_VEH  HH_MED_INC  \
CBSA  Year                                                           
10180 2002  21926.16  8121.36   3054.88           1.94   25775.875   
      2003  22846.38  8038.56   2918.50           2.47   29077.500   
      2004  23787.50  7942.75   2765.00           3.00   32452.000   
      2005  24645.00  7899.00   2680.00           

In [34]:
rail.head()

Unnamed: 0  NEW_CBSA        UPT        VRM    UPT_ADJ    VRM_ADJ  \
CBSA  Year                                                                     
10740 2009          92       NaN  1351087.0   813885.0  1351087.0   813885.0   
      2010          93       NaN  1165687.0  1415143.0  1165687.0  1415143.0   
      2011          94       NaN  1242174.0  1370680.0  1242174.0  1370680.0   
      2012          95       NaN  1129303.0  1424748.0  1129303.0  1424748.0   
      2013          96       NaN  1082588.0  1388562.0  1082588.0  1388562.0   

            VRH_ADJ  FARE_TOTAL  FARE_per_UPT  AREALANDPT   Tot_Pop  \
CBSA  Year                                                            
10740 2009  20107.0   2669729.0      1.975986   648969769  856216.0   
      2010  37249.0   2846221.0      2.441668   648969769  892014.0   
      2011  36676.0   2856263.0      2.299407   648969769  899149.0   
      2012  36870.0   2616485.0      2.316903   648969769  902794.0   
      2013  36503.0   3002928.0      2.773842   648969769  901932.0   

            Tot_Instate_Pop  Tot_Outstate_Pop  Tot_NonUSA_POP  \
CBSA  Year                                                      
10740 2009         432274.0          329891.0         11065.0   
      2010         459673.0          332367.0          9139.0   
      2011         466738.0          336918.0          9294.0   
      2012         484743.0          320910.0         12335.0   
      2013         472342.0          328386.0         11036.0   

            Total_Median_Income_Individual  Native_Instate_Med_Inc_Indiv  \
CBSA  Year                                                                 
10740 2009                         25253.0                       23216.0   
      2010                         25891.0                       23628.0   
      2011                         24924.0                       21970.0   
      2012                         25616.0                       22332.0   
      2013                         25717.0                       22709.0   

            Native_Outstate_Med_Inc_Indiv  Total_Pop_Poverty  \
CBSA  Year                                                     
10740 2009                        30017.0           846650.0   
      2010                        31267.0           881703.0   
      2011                        30564.0           887261.0   
      2012                        31553.0           887938.0   
      2013                        31723.0           890054.0   

            Pop_Below100_Poverty  Pop_Below150_Poverty  Pop_Above150_Poverty  \
CBSA  Year                                                                     
10740 2009                  15.9                  10.4                  73.7   
      2010                   NaN                   0.0                   0.0   
      2011                   NaN                   0.0                   0.0   
      2012                   NaN                   0.0                   0.0   
      2013                  19.4                  10.7                  69.8   

            Age_under18  Age_18to64  Age_over64  Total_HH  HH_0Veh   HH_1Veh  \
CBSA  Year                                                                     
10740 2009         24.8        63.1        12.1  334647.0  17776.0  120045.0   
      2010         24.4        63.2        12.4  344800.0  16484.0  120481.0   
      2011         24.4        63.0        12.6  345157.0  23644.0  121901.0   
      2012         24.1        62.7        13.2  344869.0  21014.0  120959.0   
      2013         23.7        62.5        13.8  336614.0  18513.0  118331.0   

             HH_2Veh  HH_3Veh  HH_4+Veh  PCT_HH_NO_VEH  HH_MED_INC  \
CBSA  Year                                                           
10740 2009  127430.0  47757.0   21639.0           5.31     46824.0   
      2010  136269.0  48104.0   23462.0           4.78     47383.0   
      2011  126939.0  52444.0   20229.0           6.85     45027.0   
      2012  130200.0  48806.0   23890.0    

In [36]:
bus.to_csv('temp.csv')

In [38]:
duplicates = bus[bus.duplicated()]

In [39]:
duplicates

,,Unnamed: 0,NEW_CBSA,UPT,VRM,UPT_ADJ,VRM_ADJ,VRH_ADJ,FARE_TOTAL,FARE_per_UPT,AREALANDPT,Tot_Pop,Tot_Instate_Pop,Tot_Outstate_Pop,Tot_NonUSA_POP,Total_Median_Income_Individual,Native_Instate_Med_Inc_Indiv,Native_Outstate_Med_Inc_Indiv,Total_Pop_Poverty,Pop_Below100_Poverty,Pop_Below150_Poverty,Pop_Above150_Poverty,Age_under18,Age_18to64,Age_over64,Total_HH,HH_0Veh,HH_1Veh,HH_2Veh,HH_3Veh,HH_4+Veh,PCT_HH_NO_VEH,HH_MED_INC,HH_MEAN_INC,INC_U35,INC_35_100,INC_100P,TOT_UNEMP_MSA,TOT_EMP_MSA,TOT_LABOR_MSA,UNEMP_RATE_PCT,EMP_RATE_PCT,GasPrice,Area_acre,AVG_SPEED,2018_Dollar_Multiplier,TNC_ARRIVAL,TNC_FLAG,YEARS_SINCE_TNC,PROBLEM_FLAG,dockCt,docklessCt,scooterCt,PBS_Start,PBS_End,PBS_Flag,walkscore,transit,bike,Unnamed: 0_log,NEW_CBSA_log,UPT_log,VRM_log,UPT_ADJ_log,VRM_ADJ_log,VRH_ADJ_log,FARE_TOTAL_log,FARE_per_UPT_log,AREALANDPT_log,Tot_Pop_log,Tot_Instate_Pop_log,Tot_Outstate_Pop_log,Tot_NonUSA_POP_log,Total_Median_Income_Individual_log,Native_Instate_Med_Inc_Indiv_log,Native_Outstate_Med_Inc_Indiv_log,Total_Pop_Poverty_log,Pop_Below100_Poverty_log,Pop_Below150_Poverty_log,Pop_Above150_Poverty_log,Age_under18_log,Age_18to64_log,Age_over64_log,Total_HH_log,HH_0Veh_log,HH_1Veh_log,HH_2Veh_log,HH_3Veh_log,HH_4+Veh_log,PCT_HH_NO_VEH_log,HH_MED_INC_log,HH_MEAN_INC_log,INC_U35_log,INC_35_100_log,INC_100P_log,TOT_UNEMP_MSA_log,TOT_EMP_MSA_log,TOT_LABOR_MSA_log,UNEMP_RATE_PCT_log,EMP_RATE_PCT_log,GasPrice_log,Area_acre_log,AVG_SPEED_log,2018_Dollar_Multiplier_log,TNC_ARRIVAL_log,TNC_FLAG_log,YEARS_SINCE_TNC_log,PROBLEM_FLAG_log,dockCt_log,docklessCt_log,scooterCt_log,PBS_Start_log,PBS_End_log,PBS_Flag_log,walkscore_log,transit_log,bike_log
CBSA,Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [35]:

# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ \
                    ~ VRM_ADJ \
                    + EntityEffects \
                    ',data=data2)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]


ValueError: total size of new array must be unchanged

In [23]:
# identify missing/inf values
data2 = bus[['UPT_ADJ_log', 
             'VRM_ADJ_log'
            ]]
data2 = data2.replace([np.inf, -np.inf], np.nan)
data2 = data2.dropna()
len_diff = len(bus) - len(data2)
if len_diff>0: 
    print('Excluding ' + str(len_diff) + ' records with missing values.')

# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ \
                    ~ VRM_ADJ \
                    + EntityEffects \
                    ',data=data2)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]


ValueError: total size of new array must be unchanged

In [10]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]


ValueError: total size of new array must be unchanged

In [17]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + UNEMP_RATE_PCT_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.1885
Estimator:                   PanelOLS   R-squared (Between):              0.5304
No. Observations:                4504   R-squared (Within):               0.1885
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.5272
Time:                        11:40:00   Log-likelihood                   -2182.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      243.18
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(4,4187)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             243.18
                            

In [18]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Total_HH_log \
                    + UNEMP_RATE_PCT_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.1885
Estimator:                   PanelOLS   R-squared (Between):              0.5272
No. Observations:                4504   R-squared (Within):               0.1885
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.5240
Time:                        11:40:49   Log-likelihood                   -2182.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      243.10
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(4,4187)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             243.10
                            

In [19]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + Tot_Pop_log \
                    + UNEMP_RATE_PCT_log \
                    + PCT_HH_NO_VEH_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.1886
Estimator:                   PanelOLS   R-squared (Between):              0.5301
No. Observations:                4499   R-squared (Within):               0.1886
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.5269
Time:                        11:41:43   Log-likelihood                   -2179.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      194.39
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.374   Distribution:                  F(5,4181)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             194.39
                            

In [22]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + UNEMP_RATE_PCT_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4857
Estimator:                   PanelOLS   R-squared (Between):              0.4763
No. Observations:                4504   R-squared (Within):               0.4857
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.4731
Time:                        11:43:38   Log-likelihood                   -1155.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      790.58
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(5,4186)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             790.58
                            

In [23]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4858
Estimator:                   PanelOLS   R-squared (Between):              0.8556
No. Observations:                4504   R-squared (Within):               0.4858
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.8521
Time:                        11:44:22   Log-likelihood                   -1155.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      791.01
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(5,4186)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             791.01
                            

In [27]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + TNC_FLAG \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4865
Estimator:                   PanelOLS   R-squared (Between):              0.7661
No. Observations:                4504   R-squared (Within):               0.4865
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.7623
Time:                        13:40:48   Log-likelihood                   -1151.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      660.95
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(6,4185)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             660.95
                            

In [26]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + YEARS_SINCE_TNC \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4873
Estimator:                   PanelOLS   R-squared (Between):              0.9507
No. Observations:                4504   R-squared (Within):               0.4873
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.9483
Time:                        13:40:21   Log-likelihood                   -1148.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      662.98
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(6,4185)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             662.98
                            

In [28]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + AVG_SPEED_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + YEARS_SINCE_TNC \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.5042
Estimator:                   PanelOLS   R-squared (Between):              0.9259
No. Observations:                4504   R-squared (Within):               0.5042
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.9235
Time:                        13:41:30   Log-likelihood                   -1072.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      607.96
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(7,4184)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             607.96
                            

In [29]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + YEARS_SINCE_TNC \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4873
Estimator:                   PanelOLS   R-squared (Between):              0.9507
No. Observations:                4504   R-squared (Within):               0.4873
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.9483
Time:                        13:42:46   Log-likelihood                   -1148.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      662.98
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(6,4185)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             662.98
                            

In [31]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('UPT_ADJ_log \
                    ~ VRM_ADJ_log \
                    + FARE_per_UPT_log \
                    + Tot_Pop_log \
                    + TOT_EMP_MSA_log \
                    + GasPrice_log \
                    + PCT_HH_NO_VEH \
                    + YEARS_SINCE_TNC \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\panel\data.py:39: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  new_df = df.copy().loc[self._full_index]
C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:            UPT_ADJ_log   R-squared:                        0.4879
Estimator:                   PanelOLS   R-squared (Between):              0.9593
No. Observations:                4504   R-squared (Within):               0.4879
Date:                Fri, Oct 25 2019   R-squared (Overall):              0.9572
Time:                        13:44:46   Log-likelihood                   -1146.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      569.46
Entities:                         313   P-value                           0.0000
Avg Obs:                       14.390   Distribution:                  F(7,4184)
Min Obs:                       1.0000                                           
Max Obs:                       17.000   F-statistic (robust):             569.46
                            